In [12]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

tfidf_word_value = []
X = []

readIn = pd.read_csv("../Clean/lemma_result.csv")

X_temp = readIn.iloc[:500, :-1].values
Y = readIn.iloc[:500, -1].values

for sentence in X_temp:
    X.append(sentence[0])
    
X = np.array(X)

#split data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

# TF-IDF
vectorizer = TfidfVectorizer()
df_train = vectorizer.fit_transform(X_train).toarray()
df_test = vectorizer.transform(X_test).toarray()
print(df_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [2]:
print(len(vectorizer.vocabulary_))

9218


In [13]:
# Build the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
embedding_vector_length = 32
vocab_size = len(vectorizer.vocabulary_)
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,input_length=X.shape[1]))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
print(model.summary())

IndexError: tuple index out of range

In [9]:
history = model.fit(df_train,Y_train,
                  validation_split=0.2, epochs=5, batch_size=32)

ValueError: Error when checking input: expected embedding_2_input to have shape (1,) but got array with shape (9218,)

In [ ]:
from tensorflow.keras.models import model_from_yaml

# serialize model to YAML
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")